# SPAM DETECTION


## Preparation
In this part you will set up all things

First of all, you need to import all necessary external modules.
In this scenario we will use the following libraries:<br>
tensorflow_hub  -   includes word embeddings (you also need installed tensorflow); <br>
numpy           -   is a library used for basic math and linear algebra calculations;<br>
pandas          -   provides a comfort way to work with large amounts of data through DataFrames;<br>
sklearn         -   contains a lot of different algorithms and tools useful for ML projects;<br>

In [1]:
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving Natalia_Sokyrko_dataset.txt to Natalia_Sokyrko_dataset.txt


Secondly, you need to specify the locations to all external files (for example, your dataset). You have to also specify all possible parameters for your code and store them in variables. This way, you will be protected from accidental typos :)

In [4]:
# specify location your dataset here
DATA_PATH = "Natalia_Sokyrko_dataset.txt"

# give name to label-column and text-column
COLUMN_LABEL = "label"
COLUMN_TEXT = "text"

# these are labels that indicate the type of message.
LABEL_LEGIT = 'LEGI'
LABEL_SPAM = 'SPAM'
LABEL_SMISHING = 'SMIS'

## Dataset 
In this part, we will load dataset and check it for errors.<br>
Here dataset is loaded from file into a DataFrame. DataFrame is basically a database or a table with columns where all the data is stored.<br>
You can access those columns by calling them by name:<br>
<code> column = dataframe\[column_name\] </code>

In [5]:
dataset = pd.read_csv(DATA_PATH, sep='\t', names=[COLUMN_LABEL, COLUMN_TEXT], header=None)
print('Total size:', dataset.shape[0])
print('Legit messages:', dataset[dataset[COLUMN_LABEL] == LABEL_LEGIT].shape[0])
print('Spam messages:', dataset[dataset[COLUMN_LABEL] == LABEL_SPAM].shape[0])
print('Smishing messages:', dataset[dataset[COLUMN_LABEL] == LABEL_SMISHING].shape[0])

Total size: 1336
Legit messages: 1051
Spam messages: 150
Smishing messages: 135


For now we don't realy need smishing messages, so we will remove them.

In [6]:
dataset = dataset[((dataset[COLUMN_LABEL] == LABEL_LEGIT) | (dataset[COLUMN_LABEL] == LABEL_SPAM))]

# Let's check if they are gone
print('Smishing messages:', dataset[dataset[COLUMN_LABEL] == LABEL_SMISHING].shape[0])

Smishing messages: 0


If the dataset is fine, you may proceed to next step

## Data preprocessing
The difference between humans and machines is that the machines don't know words. So it is very difficult for them to process human language.<br>
To help them, scientists invented a bunch of methods to translate words and sentences into sets of numbers, which machines are used to. Those methods are called "embeddings".<br>
The point of embedding is to translate words (or complete sentences) into a new space of features (usualy of high dimensionality), so every word (sentence) will be represented as vector in this new space.<br>
In this example we will use ELMO embeddings (but you can use something else if you wish).<br>
You can also use simpler approach and transform your messages into feature-vectors using features of the message itself (for example, you can count length of every message as a feature, or you can count number of URLs).<br>
The more different features you use, the better are chances to succeed. 

In [7]:
def messages2vectors(messages):
    '''
    Transforms single message into feature-vector;
    Parameters:
        messages    -   array of strings;
    Returns:
        features    -   array of feature-vectors;   
    '''

    elmo = hub.Module("https://tfhub.dev/google/elmo/1")

    features = np.zeros((0, 1024))
    n = 100
    l = int(len(messages) / n) if len(messages) % n == 0 else int(len(messages) / n) + 1
    for i in range(l):

        if (i + 1) * n < len(messages):
            right = (i + 1) * n
            embedds = elmo(messages[int(i * n) : right], signature="default", as_dict=True)["default"] 
        else:
            embedds = elmo(messages[:len(messages) - int(i * n)], signature="default", as_dict=True)["default"] 

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            embedds = sess.run(embedds)
            features = np.concatenate([features, embedds])

    return features

Next step is to prepare labels. You need to convert all 'LEGI' labels to 0s and all 'SPAM' labels to 1s.<br>
For example: \['LEGI', 'LEGI', 'SPAM', 'LEGI', 'SPAM'\] -> \[0, 0, 1, 0, 1\]

In [8]:
def convert_labels(labels_raw):
    '''
    Transforms labels into numerical values;
    Parameters:
        labels_raw    -   array of text-labels;
    Returns:
        features    -   array of numerical labels;   
    ''' 

    # add your code here
    labels = np.asarray([0 if i=='LEGI' else 1 for i in labels_raw ]).astype('int')

    return labels

Now let's transform messages to features and change labels to numerical values

In [9]:
features = messages2vectors(dataset[COLUMN_TEXT])
labels = convert_labels(dataset[COLUMN_LABEL])
print(features.shape)
print(labels.shape)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(1201, 1024)
(1201,)


To ensure that our model's predictions are valid, we need to split our data into 2 parts: training and testing.<br>
Because of it's nature, ML-algorithms have tendency to overfit on training data (which means that algorithm no longer 'predicts' class based on input but rather remembers that a particular input corresponds to a particular output).<br>
Overfitting causes model to completely fail on any data not present in training set.<br>
Separate (independent from training) data provides us with unbiased results to see 'real' performance.

In [11]:
def split_data(features, labels, ratio=0.7):
    '''
    Splits dataset into train/test parts using given ratio;
    Parameters:
        data    -   array of features;
        labels  -   array of corresponding labels;
        ratio   -   train/test size ratio;
    Returns:
        train_data      -   array of training features;   
        train_labels    -   array of training labels; 
        test_data       -   array of testing features; 
        test_labels     -   array of testing labels; 
    '''    


    positive_data = features[labels == 1] # all spam features
    negative_data = features[labels == 0] # all legit features

    # We shuffle arrays to get random samples later
    random_indecies_positive = np.arange(positive_data.shape[0])
    np.random.shuffle(random_indecies_positive)
    random_indecies_negative = np.arange(negative_data.shape[0])
    np.random.shuffle(random_indecies_negative)

    n_positive_train = int(positive_data.shape[0] * ratio)
    n_negative_train = int(negative_data.shape[0] * ratio)

    # Training data are all indecies in 'ratio' part of shuffled indecies
    train_data = np.concatenate([positive_data[random_indecies_positive[:n_positive_train]], 
                                negative_data[random_indecies_negative[:n_negative_train]]])
    
    train_labels = np.asarray([1] * n_positive_train + [0] * n_negative_train)

    # Testing data are all indecies that remain
    test_data = np.concatenate([positive_data[random_indecies_positive[n_positive_train:]], 
                                negative_data[random_indecies_negative[n_negative_train:]]])

    test_labels = np.asarray([1] * (positive_data.shape[0]  - n_positive_train) + [0] * (negative_data.shape[0] - n_negative_train))

    return train_data, train_labels, test_data, test_labels



## Metrics
To see how good (or bad) our spam detector works, we have to use some metrics. In assigment you are required to compare FAR and FRR of different algorithms.<br>
FAR (False Acceptance Rate) - ratio of positive samples (spam in our case) wrongly predicted as negative (legitimate);<br>
FRR (False Rejection Rate) - ratio of negative samples (legitimate) wrongly predicted as positive (spam);<br>
These rates represent False Negative and False Positive Errors (you may also know them by names Type-1 Error and Type-2 Error)

Your task is to compute FAR and FRR based on given true labels and predicted labels

In [12]:
def get_metrics(labels, predictions):
    '''
    Computes metrics;
    Parameters:
        labels    -   array of labels;
        predictions  -   array of predictions;
    Returns:
        FAR -   False Acceptance Rate;
        FRR -   False Rejection Rate;
    '''  
    # add your code here
    # FAR = FPR = FP/(FP+TN)
    # FRR = FNR = FN/(TP+FN)
    conf_matr = metrics.confusion_matrix(labels, predictions)
    TN = conf_matr[0][0]
    FP = conf_matr[0][1]
    FN = conf_matr[1][0]
    TP = conf_matr[1][1]
    FAR = FP / (FP+TN)
    FRR = FN / (TP+FN)

    return FAR, FRR

## Model initialization
In this part we will create classifier and set it up. We will use Random Forest as example (note that in your assigment you must use all of given algorithms).<br>
Note, that every algorithm has it's unique set of parameters (called hyperparameters).<br>
Also, algorithms from <code>sklearn</code> library usualy have common methods <code>fit(X, Y)</code> and <code>predict(X)</code>.

In [13]:
classifierType = sklearn.ensemble.RandomForestClassifier
hyperparameters = {'n_estimators' : 100,
                'criterion' : 'gini',
                'max_depth' : None,
                'min_samples_split' : 2}

## Model Training and evaluation
Complete method below to do the following:<br>
1) Split <code>data</code> an labels into training and testing sets;<br>
2) Fit your model on training set using <code>fit(X, Y)</code> method;<br> 
3) Make predictions based on training set using <code>predict(X)</code> method;<br>
4) Compute FAR/FRR for training set;<br>
5) Make predictions based on testing set;<br>
6) Compute FAR/FRR for testing set;<br>


In [14]:
def evaluate(classifierType, hyperparameters, features, labels):
    '''
    Splits dataset into train/test parts using given ratio;
    Parameters:
        classifierType      -   type of ML algorithm to use;
        hyperparameters     -   dictionary of model's parameters;
        features            -   array of features;
        labels              -   array of labels
    Returns:
        trainFAR    -   False Acceptance Rate for train dataset;
        trainFRR    -   False Rejection Rate for train dataset;
        testFAR     -   False Acceptance Rate for test dataset;
        testFRR    -   False Rejection Rate for test dataset;
    '''    

    model = classifierType(**hyperparameters)

    # Split data
    train_data, test_data, train_labels, test_labels = train_test_split(features, labels)

    print('Train set shape:', train_data.shape)
    print('Train labels shape:', train_labels.shape)
    print('Test set shape:', test_data.shape)
    print('Test labels shape:', test_labels.shape)

    # Fit your model
    model.fit(train_data, train_labels)


    # Make predictions for training dataset
    predictions = model.predict(train_data)


    # Compute train FAR/FRR
    # add your code here
    trainFAR, trainFRR = get_metrics(train_labels, predictions)

    # Make predictions for testing dataset
    # add your code here
    predictions_test = model.predict(test_data)

    # Compute test FAR/FRR
    # add your code here
    testFAR, testFRR = get_metrics(test_labels, predictions_test)

    return trainFAR, trainFRR, testFAR, testFRR

In [17]:
# Check if it works :)
trainFAR, trainFRR, testFAR, testFRR = evaluate(classifierType, hyperparameters, features, labels)
print('Train:')
print('\tFAR:', trainFAR)
print('\tFRR:', trainFRR)

print('Test:')
print('\tFAR:', testFAR)
print('\tFRR:', testFRR)

Train set shape: (900, 1024)
Train labels shape: (900,)
Test set shape: (301, 1024)
Test labels shape: (301,)
Train:
	FAR: 0.0
	FRR: 0.0
Test:
	FAR: 0.0149812734082397
	FRR: 0.5294117647058824


## Final Task
Combine your knowledge and compute FAR/FRR metrics for other algorithms from assignment.<br>
Don't forget to fill report and share it on google class.<br>
Good luck :)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(features, labels)

# KNeighborsClassifier

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=2, algorithm='ball_tree')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[259   5]
 [ 10  27]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       264
           1       0.84      0.73      0.78        37

    accuracy                           0.95       301
   macro avg       0.90      0.86      0.88       301
weighted avg       0.95      0.95      0.95       301



In [21]:
trainFAR, trainFRR = get_metrics(y_train, classifier.predict(X_train))
testFAR, testFRR = get_metrics(y_test, classifier.predict(X_test))
print('Train:')
print('\tFAR:', trainFAR)
print('\tFRR:', trainFRR)

print('Test:')
print('\tFAR:', testFAR)
print('\tFRR:', testFRR)

Train:
	FAR: 0.0
	FRR: 0.1504424778761062
Test:
	FAR: 0.01893939393939394
	FRR: 0.2702702702702703


# SVM

In [22]:
from sklearn import svm
clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True, tol=0.0001, C=100.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=100.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
          penalty='l2', random_state=None, tol=0.0001, verbose=0)

In [23]:
trainFAR, trainFRR = get_metrics(y_train, clf.predict(X_train))
testFAR, testFRR = get_metrics(y_test, clf.predict(X_test))
print('Train:')
print('\tFAR:', trainFAR)
print('\tFRR:', trainFRR)

print('Test:')
print('\tFAR:', testFAR)
print('\tFRR:', testFRR)

Train:
	FAR: 0.0
	FRR: 0.0
Test:
	FAR: 0.03787878787878788
	FRR: 0.10810810810810811
